In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import random

# Reproducibility
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [2]:
# Load preprocessed data
X_drug = np.load("X_drug.npy")       # shape (samples, 1024)
X_protein = np.load("X_protein.npy") # shape (samples, 20000)
y = np.load("y.npy")                 # shape (samples, 1)

print("Drugs:", X_drug.shape)
print("Proteins:", X_protein.shape)
print("Labels:", y.shape)


Drugs: (30056, 1024)
Proteins: (30056, 20000)
Labels: (30056, 1)


In [3]:
class DTADataset(Dataset):
    def __init__(self, X_drug, X_protein, y):
        self.X_drug = torch.tensor(X_drug, dtype=torch.float32)
        self.X_protein = torch.tensor(X_protein, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X_drug[idx], self.X_protein[idx], self.y[idx]


In [4]:
dataset = DTADataset(X_drug, X_protein, y)

# 80-20 split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Train size: {len(train_dataset)}, Test size: {len(test_dataset)}")


Train size: 24044, Test size: 6012


In [5]:
class DeepDTA(nn.Module):
    def __init__(self):
        super(DeepDTA, self).__init__()

        # Drug CNN
        self.drug_cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten()
        )

        # Protein CNN
        self.protein_cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten()
        )

        # Fully Connected Layers
        self.fc = nn.Sequential(
            nn.Linear(64*252 + 64*4996, 1024), # adjust sizes dynamically
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1)  # regression output
        )

    def forward(self, drug, protein):
        # Reshape to (batch, channel=1, length)
        drug = drug.unsqueeze(1)
        protein = protein.unsqueeze(1)

        drug_feat = self.drug_cnn(drug)
        protein_feat = self.protein_cnn(protein)

        combined = torch.cat((drug_feat, protein_feat), dim=1)
        output = self.fc(combined)
        return output


In [7]:
class DeepDTA(nn.Module):
    def __init__(self):
        super(DeepDTA, self).__init__()

        # Drug CNN
        self.drug_cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten()
        )

        # Protein CNN
        self.protein_cnn = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Conv1d(32, 64, kernel_size=8, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(2),
            nn.Flatten()
        )

        # 🔹 Dynamically determine feature size
        with torch.no_grad():
            dummy_drug = torch.zeros(1, 1, 1024)     # (batch=1, channel=1, drug_len=1024)
            dummy_protein = torch.zeros(1, 1, 20000) # (batch=1, channel=1, prot_len=20000)

            drug_feat = self.drug_cnn(dummy_drug)
            protein_feat = self.protein_cnn(dummy_protein)

            combined_dim = drug_feat.shape[1] + protein_feat.shape[1]
            print("✅ Combined feature dim:", combined_dim)

        # Fully Connected Layers
        self.fc = nn.Sequential(
            nn.Linear(combined_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 1)
        )

    def forward(self, drug, protein):
        drug = drug.unsqueeze(1)       # (batch, 1, 1024)
        protein = protein.unsqueeze(1) # (batch, 1, 20000)

        drug_feat = self.drug_cnn(drug)
        protein_feat = self.protein_cnn(protein)

        combined = torch.cat((drug_feat, protein_feat), dim=1)
        output = self.fc(combined)
        return output


In [8]:
model = DeepDTA().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_drug, batch_protein, batch_y in train_loader:
        batch_drug, batch_protein, batch_y = batch_drug.to(device), batch_protein.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_drug, batch_protein)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * batch_drug.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")


✅ Combined feature dim: 335616
Epoch 1/10, Loss: 22345942.5922
Epoch 2/10, Loss: 16501151.9162
Epoch 3/10, Loss: 15356083.5187
Epoch 4/10, Loss: 13440187.4793
Epoch 5/10, Loss: 12604278.0251
Epoch 6/10, Loss: 12121851.4896
Epoch 7/10, Loss: 11666248.0245
Epoch 8/10, Loss: 11350930.7425
Epoch 9/10, Loss: 11033974.7203
Epoch 10/10, Loss: 10766962.1280


In [9]:
model.eval()
mse, total = 0, 0
with torch.no_grad():
    for batch_drug, batch_protein, batch_y in test_loader:
        batch_drug, batch_protein, batch_y = batch_drug.to(device), batch_protein.to(device), batch_y.to(device)
        preds = model(batch_drug, batch_protein)
        mse += criterion(preds, batch_y).item() * batch_drug.size(0)
        total += batch_drug.size(0)

test_rmse = np.sqrt(mse / total)
print("✅ Test RMSE:", test_rmse)


✅ Test RMSE: 3239.7460288327115


In [10]:
model.eval()
with torch.no_grad():
    drug_sample = torch.rand(1, 1024).to(device)     # Example random drug fingerprint
    protein_sample = torch.rand(1, 20000).to(device) # Example random protein sequence encoding

    pred = model(drug_sample, protein_sample)
print("Predicted Binding Affinity:", pred.item())


Predicted Binding Affinity: 1848.3349609375


In [11]:
torch.save(model.state_dict(), "deepdta_model.pth")

# Later load it
model = DeepDTA().to(device)
model.load_state_dict(torch.load("deepdta_model.pth"))
model.eval()


✅ Combined feature dim: 335616


DeepDTA(
  (drug_cnn): Sequential(
    (0): Conv1d(1, 32, kernel_size=(8,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(32, 64, kernel_size=(8,), stride=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (protein_cnn): Sequential(
    (0): Conv1d(1, 32, kernel_size=(8,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(32, 64, kernel_size=(8,), stride=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
  )
  (fc): Sequential(
    (0): Linear(in_features=335616, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): ReLU()
    (5

In [12]:
# Suppose you have a new drug fingerprint (size=1024)
custom_drug = np.random.rand(1024)
custom_protein = np.random.rand(20000)

drug_tensor = torch.tensor(custom_drug, dtype=torch.float32).unsqueeze(0).to(device)
protein_tensor = torch.tensor(custom_protein, dtype=torch.float32).unsqueeze(0).to(device)

with torch.no_grad():
    pred = model(drug_tensor, protein_tensor)

print("Predicted Binding Affinity:", pred.item())


Predicted Binding Affinity: 3924.860595703125
